In [4]:
import networkx as nx
import numpy as np
from copy import deepcopy

##########################################

def inverse_hcat(graph,degrees):
  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]
  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T
  M=np.diag(d)
  return d+np.matmul(M,np.matmul(adj,d))

def inverse_card(graph,degrees):
  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]

  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T
  att=np.sum(adj,axis=1)
  att_m=np.diag(np.sum(adj,axis=1))
  div=np.zeros(len(d))
  np.divide(np.matmul(np.diag(d),np.matmul(adj,d)),att,where=att!=0,out=div)
  lhs=d+np.matmul(att_m,d)+div
  return lhs

def inverse_mbs(graph,degrees):
  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T

  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]
  M=np.diag(d)

  return d+np.matmul(M,np.max(np.matmul(adj,M),axis=1))

##########################################
class CAF:
    def __init__(self,af,intervals):
        self.af = af #this is a networkx graph
        self.intervals = intervals #this is a dictionary of min/max tuples with arguments as keys
    
    def is_rational(self,semantics):
        #semantics is one of hcat, card, mbs
        #a CAF is irrational if there is no weighting s.t. sigma(a_i) \in I(a_i) for all a_i
        #so we need to do the inverse semantics thing and check whether all arguments are in the interval.
        #we can do this by checking whether the minimum of the CAF is valid (assuming axial-radiality)
        deg = [0] * len(self.af.nodes)
        for i in range(len(self.af.nodes)):
           deg[i]=min(self.intervals[i])
        weights = semantics(self.af,deg)
        if max(weights)>1:
            return False
        else:
            return True
    
    def is_fully_rational(self,semantics):
       #as above, but check max interval
        deg = [0] * len(self.af.nodes)
        for i in range(len(self.af.nodes)):
           deg[i]=max(self.intervals[i])
        weights = semantics(self.af,deg)
        if max(weights)>1:
            return False
        else:
            return True
      
    def is_epsilon_rational(self,semantics,epsilon):
      for i in range(len(self.af.nodes)):
          if not epsilon <= max(self.intervals[i])-min(self.intervals[i]):
             return False
          new_caf = deepcopy(self)
          new_caf.intervals[i]=(self.intervals[i][1]-epsilon,self.intervals[i][1])
          if not new_caf.is_rational(semantics):
              return False
      return True
    
    def best_refinement(self,semantics,epsilon):
        for a in self.arguments:
            new_caf=deepcopy(self)
            new_caf.intervals[a]=(self.intervals[a][1],self.intervals[a][1])
            if not new_caf.is_rational(semantics):
                l = min(self.intervals[a])
                r = max(self.intervals[a])
                found = False
                while not found:
                   m = (l+r)/2
                   new_caf2 = deepcopy(new_caf)
                   new_caf2.intervals[a]=(m,m)
                   if not new_caf2.is_rational(semantics):
                      r = m
                      if (l-2/2)<epsilon:
                         self.intervals[a]=(min(self.intervals[a]),m)
                         found = True
                   else:
                      l = m

In [5]:
NUM_TRIALS = 100

def make_random_graph(size):
    g = nx.fast_gnp_random_graph(size,0.5)
    intervals = {}
    for i in range(size):
        intervals[i]=(0,1)
    return CAF(g,intervals)

for i in range(NUM_TRIALS):
    #generate a random CAF
    caf = make_random_graph(10)
    while not caf.is_rational(inverse_hcat) and caf.is_epsilon_rational(inverse_hcat,0.01):
        caf = make_random_graph(10)
        caf.best_refinement(inverse_hcat,0.01)
        if not caf.is_epsilon_rational(inverse_hcat,0.01):
            print("Error")
            break




IndexError: list assignment index out of range